In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf

In [ ]:
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [ ]:
train = pd.read_csv('../input/train.csv', header=None)


In [ ]:
train.head()


In [ ]:
test = pd.read_csv('../input/test.csv', header=None)


In [ ]:
test.drop(test.columns[0:5],axis=1, inplace=True)
test.head()

In [ ]:
test.columns=test.iloc[0]

In [ ]:
test1=test.drop(0)
test1.head()


In [ ]:
test1=test1.astype(float)
test1.info()

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')
sample.head()

In [ ]:
train.columns=train.iloc[0]
train.head()

In [ ]:
train1=train.drop(0)
train1.head()

In [ ]:
train1.info()

In [ ]:
train1.drop(train1.columns[0:3], axis=1, inplace=True)

In [ ]:
train1.head()

In [ ]:
train1.info()

In [ ]:
train1.describe()

In [ ]:
train2=train1.astype(float)

In [ ]:
train2.info()

In [ ]:
train2.describe()

In [ ]:
import seaborn as sns                 
import matplotlib.pyplot as plt
matrix = train2.corr()
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(matrix, vmax=.8, square=True, cmap="BuPu");

In [ ]:
X=train2.drop(columns=["bestSoldierPerc"])
X.abs()
X.info()

In [ ]:
y=train2["bestSoldierPerc"]
print(y[y==1.0].value_counts())
print(y[y==0.0].value_counts())
y.astype(float)

In [ ]:
#y[y<1]=0
#print(y[y==1].value_counts())
#print(y[y==0].value_counts())
y.head()
y=y*100


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [ ]:

test2=sc.fit_transform(test1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Activation


In [ ]:
n_cols = X.shape[1]
model = Sequential()
#add model layers
model.add(Dense(1024, activation='relu', input_shape=(n_cols,)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(256 , activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(1))

model.add(Activation("linear"))


model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_absolute_error',metrics= ['mean_absolute_error'])



In [ ]:
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=15)
#train model
model.fit(X, y, validation_split=0.2, epochs=20,shuffle=True,batch_size=1024,callbacks=[early_stopping_monitor])


In [ ]:
y_pred= model.predict(test2)

In [ ]:
y_pred=y_pred/100

In [ ]:
df=pd.DataFrame(y_pred,index=np.arange(len(y_pred)),columns=["bestSoldierPerc"])
df.head()

In [ ]:
sample["bestSoldierPerc"]=df

In [ ]:
sample.to_csv("sample_submission.csv")

In [ ]:
sample.shape

In [ ]:
sample